## **Workflow to compute Normalised Difference Trophic Index (NDTrI) with OpenEO datacube**

#### ***Install and import libraries***

In [1]:
# To install libraries:

# Windows-Miniconda (takes huge amount of time for 'resolving environment')
# conda create -n myenv python (you need to specify python directly)
# conda install -c conda-forge rioxarray openeo geopandas leafmap netCDF4 folium matplotlib mapclassify rasterio openeo leafmap localtileserver
# openeo and leafmap are available through conda-forge channel only
# 5. conda install conda-forge::localtileserver
# 2. conda install -c conda-forge gdal
# 3. conda install -c conda-forge rasterio (because rasterio built on gdal): https://stackoverflow.com/questions/70208030/fixing-dll-load-failed-the-specified-module-could-not-be-found-error-in-pyth

# Ubuntu: !pip install rioxarray openeo geopandas leafmap netCDF4 'folium' 'matplotlib' 'mapclassify'
# pip install rioxarray openeo geopandas leafmap netCDF4 folium matplotlib mapclassify
# pip install localtileserver if you face issues

# conda install rioxarray geopandas netCDF4 folium matplotlib mapclassify rasterio
# conda install -c conda-forge leafmap openeo

In [2]:
# platform libraries
import openeo

# utility libraries
from datetime import date
import numpy as np
import xarray as xr
import rioxarray
import json
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import leafmap.foliumap as leafmap

import os

#### ***Define the configuration:***

For this workflow, only one input dataset is required, representing the boundaries of water body (geopackage format recommended).
Below the following configuration is defined:
- paths and names of files (input, output, temporary)
- datacube parameters (such as cloud filtering, while others are given in the next cells)
- processing parameters (batch jobs and width of buffer to exclude shore areas)
- visualisation

In [20]:
# PATHS and NAMES
input_dir = 'data/input'
temp_dir = 'data/temp'
output_dir = 'data/output'

vector = gpd.read_file(f"{input_dir}/osm_water.gpkg")
vector
feature_name_col = "name" # name of column describing the feature name in input vector dataset
feature_name = "Groß Glienicker See" # feature name (lake)

# define raster filenames (temp and outpur)
filenames = {
    'full_nc': 'full.nc',
    'full_nc_remap': 'full_remap.nc',
    'full_tif': 'full.tif',
    'cloud_nc': 'cloud.nc',
    'cloud_tif': 'cloud.tif',
    'water_tif': 'water.tif',
    'water_nc': 'water.nc',
    'masked_cloud_nc': 'masked.nc',  # to mask  clouds (SCL 7-11)
    'masked_cloud_tif': 'masked.tiff',
    'masked_water_nc': 'masked_water.nc',  # to mask out non-water (SCL != 6)
    'masked_water_tif': 'masked_water.tif',
    'test_nc': 'test.nc',  # NDTrI
    'test_tif': 'test.tif', # NDTrI
    'ndtri_tif': 'ndtri_result.tif',
    'masked_ndtri_tif': 'ndtri_masked.tif',
    'timeseries': 'timeseries.json',
    'full_tif_reproj': 'full_reproj.tif',
    'masked_cloud_tif_reproj': 'masked_reproj.tif',
    'masked_water_tif_reproj': 'masked_water_reproj.tif',
    'test_tif_reproj': 'test_reproj.tif'
} # TODO - divide temporary and output

file_paths = {key + '_path': os.path.join(temp_dir, filename) for key, filename in filenames.items()} # generate paths
globals().update(file_paths) # unpack variables
for key, value in file_paths.items():
    print(f"{key}: {value}")

# DATACUBE PARAMETERS
overall_cloud = False # if True, filter by the cloud percentage in the properties of datacube (for the whole datacube)
cloud_percentage = 10 # maximum allowed share of cloud pixels

# PROCESSING PARAMETERS
shore_negative_buffer = 2 # 'False' or Integer to calculate negative buffer from the lake borderline
batch = False # if False, download datacube directly from OpenEO. Otherwise, submit a batch job to be completed on the backend.

# VISUALISATION
leaflet_offset = 0.001
zoom = 14

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = "/proxy/9999" # to enable localtileserver to visualise raster layers on Leafmap
# os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = f"{os.environ['JUPYTERHUB_SERVICE_PREFIX']}/proxy/9999"

full_nc_path: data/temp/full.nc
full_nc_remap_path: data/temp/full_remap.nc
full_tif_path: data/temp/full.tif
cloud_nc_path: data/temp/cloud.nc
cloud_tif_path: data/temp/cloud.tif
water_tif_path: data/temp/water.tif
water_nc_path: data/temp/water.nc
masked_cloud_nc_path: data/temp/masked.nc
masked_cloud_tif_path: data/temp/masked.tiff
masked_water_nc_path: data/temp/masked_water.nc
masked_water_tif_path: data/temp/masked_water.tif
test_nc_path: data/temp/test.nc
test_tif_path: data/temp/test.tif
ndtri_tif_path: data/temp/ndtri_result.tif
masked_ndtri_tif_path: data/temp/ndtri_masked.tif
timeseries_path: data/temp/timeseries.json
full_tif_reproj_path: data/temp/full_reproj.tif
masked_cloud_tif_reproj_path: data/temp/masked_reproj.tif
masked_water_tif_reproj_path: data/temp/masked_water_reproj.tif
test_tif_reproj_path: data/temp/test_reproj.tif
[local_process] Total execution time (cumulative): 165.26 seconds


## **1. Connect to a cloud platform**
Let's connect to the Copernicus Dataspace Ecosystem. Being connected allows for data discovery.

**Note:** Endpoints may vary significantly (depending on data collections), and connection configuration should be updated for each of them.

In [4]:
conn = openeo.connect('https://openeo.dataspace.copernicus.eu/')
# TODO - implement retries with delay to avoid issue described below 

Let's authethicate through the provided URL. Being logged in allows to use the full range of functionality including processing!

**Note:** sometimes, the line above gives you error 403, but in this case you should run it again. It happens mostly when you restart Jupyter Notebook or your machine and running non-cached lines.

In [5]:
conn.authenticate_oidc()

Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

Check if the login worked:

In [6]:
conn.describe_account()

{'info': {'oidc_userinfo': {'email': 'gojd31iris@gmail.com',
   'email_verified': True,
   'family_name': 'K',
   'given_name': 'Vitalii',
   'name': 'Vitalii K',
   'preferred_username': 'gojd31iris@gmail.com',
   'sub': '030a29d0-a7c8-4057-bff5-beec0daba759'}},
 'name': 'Vitalii K',
 'user_id': '030a29d0-a7c8-4057-bff5-beec0daba759'}

Common **issue** is being disconnected without response, presumably because of hitting limit rate/expired timeout. For the details, see [here](log/issue_token_exp.txt).
No reliable debugging solution was found, only wait for an uncertain amount of time (hours).

##### ***Area of Interest***

Our region of interest can be any inland water body, but in this Notebook we will explore urban lakes in Berlin.

Sample input dataset currently derived from Open Street Map (through Overpass Turbo API, see [the query](overpass_query.txt) for details) and represents polygons of lakes.

To visualise the vector dataset, we should reproject it. We are currently interested in one lake from the input dataset, named Groß Glienicker See.

**TODO** - to replace with another name

In [7]:
import geopandas as gpd
from shapely.geometry import box
import folium

if vector.crs != ("EPSG:4326"):
    print ("CRS different from EPSG:4326. Reprojecting...")
    vector = vector.to_crs("EPSG:4326")

# filtering vector dataset by the name of lake 
vector = vector[vector[feature_name_col] == feature_name] #https://stackoverflow.com/questions/45990001/forcing-pandas-iloc-to-return-a-single-row-dataframe/45990057
vector_polygon = vector['geometry'].iloc[0].convex_hull
bounding_box = box(*vector.total_bounds) # create bbox

center_lat = float(vector.geometry.centroid.y.mean()) # better to use y.mean() and x.min()) instead of y and x if multiple features used, otherwise it will return Geoseries, not single points
center_lon = float(vector.geometry.centroid.x.mean())
center = (center_lat + leaflet_offset, center_lon + leaflet_offset)
m = leafmap.Map(center=center, zoom=zoom)

vector.convex_hull.explore(m=m)

bounding_box_df = gpd.GeoDataFrame([[bounding_box]], columns=['geometry'], crs = vector.crs)
bounding_box_df.explore(m=m, style_kwds=dict(color="black", fill = False))

m # show

##### ***Inspect Metadata***
We need to set the following configurations to define the content of the data cube we want to access:
- dataset name
- band names
- time range
- the area of interest specifed via bounding box coordinates
- spatial resolution

To select the correct dataset we can first list all the available datasets.

In [8]:
print(conn.list_collection_ids())

['SENTINEL3_OLCI_L1B', 'SENTINEL3_SLSTR', 'SENTINEL_5P_L2', 'COPERNICUS_VEGETATION_PHENOLOGY_PRODUCTIVITY_10M_SEASON1', 'COPERNICUS_VEGETATION_PHENOLOGY_PRODUCTIVITY_10M_SEASON2', 'COPERNICUS_PLANT_PHENOLOGY_INDEX', 'ESA_WORLDCOVER_10M_2020_V1', 'ESA_WORLDCOVER_10M_2021_V2', 'COPERNICUS_VEGETATION_INDICES', 'SENTINEL2_L1C', 'SENTINEL2_L2A', 'SENTINEL1_GRD', 'COPERNICUS_30', 'LANDSAT8_L2', 'SENTINEL3_SYN_L2_SYN', 'SENTINEL3_SLSTR_L2_LST', 'SENTINEL1_GLOBAL_MOSAICS']


We want to use the Sentinel-2 L2A product. It's name is `'SENTINEL2_L2A'`. 

We get the metadata for this collection as follows. This is an important step to familiarize yourself with the data collection (e.g. learn the band names). Take a note that bands vary in spatial resolution (GSD) which is common for Sentinel products.

**Note:** there is no information on **no data values** in the OpenEO collections!

In [9]:
conn.describe_collection("SENTINEL2_L2A")

{'assets': {'thumbnail': {'href': 'https://www.esa.int/var/esa/storage/images/esa_multimedia/images/2022/07/tallinn_estonia/24383281-1-eng-GB/Tallinn_Estonia_pillars.jpg',
   'roles': ['thumbnail'],
   'title': 'Sentinel 2 L2A',
   'type': 'image/png'}},
 'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['B01',
    'B02',
    'B03',
    'B04',
    'B05',
    'B06',
    'B07',
    'B08',
    'B8A',
    'B09',
    'B11',
    'B12',
    'WVP',
    'AOT',
    'SCL',
    'sunAzimuthAngles',
    'sunZenithAngles',
    'viewAzimuthMean',
    'viewZenithMean']},
  't': {'extent': ['2015-07-04T00:00:00Z', None], 'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180, 180],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-56, 83],
   'reference_system': {'$schema': 'https://proj.org/schemas/v0.2/projjson.schema.json',
    'area': 'World',
    'bbox': {'east_longitude': 180,
     'north_latitude': 90,
     'south_latitude': -90,
     'west_longitude': -180},
    'coordinate_system': {'axis': [{'abbreviation': 'Lat',
       'direction': 'north',
       'name': 'Geodetic latitude',
       'unit': 'degree'},
      {'abbreviation': 'Lon',
       'direction': 'east',
       'name': 'Geodetic longitude',
       'unit': 'degree'}],
     'subtype': 'ellipsoidal'},
    'datum': {'ellipsoid': {'inverse_flattening': 298.257223563,
      'name': 'WGS 84',
      'semi_major_axis': 6378137},
     'name': 'World Geodetic System 1984',
     'type': 'GeodeticReferenceFrame'},
    'id': {'authority': 'OGC', 'code': 'Auto42001', 'version': '1.3'},
    'name': 'AUTO 42001 (Universal Transverse Mercator)',
    'type': 'GeodeticCRS'},
   'step': 10,
   'type': 'spatial'}},
 'description': 'SENTINEL-2 is a wide-swath, high-resolution, multi-spectral imaging mission, supporting Copernicus Land Monitoring studies, including the monitoring of vegetation, soil and water cover, as well as observation of inland waterways and coastal areas. The SENTINEL-2 Multispectral Instrument (MSI) samples 13 spectral bands: four bands at 10 metres, six bands at 20 metres and three bands at 60 metres spatial resolution.\n The level 2A data is atmospherically corrected using Sen2Cor.',
 'extent': {'spatial': {'bbox': [[-180, -56, 180, 83]]},
  'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}},
 'id': 'SENTINEL2_L2A',
 'keywords': ['COPERNICUS',
  'ESA',
  'Orthoimagery',
  'Sentinel-2',
  'MSI',
  'Level-2A',
  'Radiometry',
  'Plant Resource',
  'TOC'],
 'license': 'proprietary',
 'links': [{'href': 'https://sentinels.copernicus.eu/documents/247904/690755/Sentinel_Data_Legal_Notice',
   'rel': 'license',
   'type': 'application/pdf'},
  {'href': 'https://documentation.dataspace.copernicus.eu/Data/SentinelMissions/Sentinel2.html#sentinel-2-level-2a-top-of-canopy-toc',
   'rel': 'alternate',
   'type': 'application/html'},
  {'href': 'https://dataspace.copernicus.eu/browser/?zoom=11&lat=45.36638&lng=12.49832&themeId=DEFAULT-THEME&visualizationUrl=https%3A%2F%2Fsh.dataspace.copernicus.eu%2Fogc%2Fwms%2F28b654e7-8912-4e59-9e58-85b58d768b3a&datasetId=S2

#### **2. Define the data cube**

Check the bounding box:

In [10]:
vector.total_bounds

array([13.1056403, 52.4567457, 13.118754 , 52.4739705])

Specify the parameters of data cubes:

In [11]:
from openeo.processes import lte #less than equal function

collection      = 'SENTINEL2_L2A'
spatial_extent  = {'west':vector.total_bounds[0],
                   'east':vector.total_bounds[2],
                   'south':vector.total_bounds[1],
                   'north':vector.total_bounds[3],
                   'crs':4326}
bands           = ['B02', 'B05', 'SCL']
properties = {"eo:cloud_cover": lambda x: lte(x, cloud_percentage)} # to filter out images with high cloud percentage
temporal_extent = ['2020-04-01', '2020-10-31'] # April-October for Normalised Difference Trophic Index


# TODO - to implement cloud percentage filtering by bounding box, not by timestamp properties! Otherwise, we lose some images

## ***Processing locally***
In the next chapters, we will access data from data cubes but process them **locally**.

#### 3. Load the data cube
Let's start to calculate time for processing. A module below will start to calculate execution time and show the cumulative time spent after each cell in the Notebook until processing finished.

In [12]:
import utils.track_time as track_time
track_time.register_time("local_process")

# To stop calculating time for process:
# track_time.stop_time("local_process")

Tracking execution time for process: local_process


We have defined the parameters of datacube we are interested in. Now we use these definitions to load the data cube. The earlier we apply the filtering parameters of datacube, the faster follow-up processing will be, as slices of datacube needed will be processed only.

In [13]:
load_params = {
    "spatial_extent": spatial_extent,
    "bands": bands,
    "temporal_extent": temporal_extent,
}
if overall_cloud: # if filtering by properties of collection
    load_params['properties'] = properties

s2 = conn.load_collection(collection, **load_params) # unpack key-value dictionary as a pair of parameter-value
# TODO - to implement overall_cloud condition

[local_process] Total execution time (cumulative): 0.09 seconds


**Optional:** you can run `s2.metadata` to see if the spatial resolution (GSD) is the same it was when we described the collection. Nothing changed yet!

In [14]:
# s2.metadata

[local_process] Total execution time (cumulative): 0.09 seconds


Let's download the datacube to the local machine in netcdf and geotiff formats. 
From the OpenEO documentation it is unclear how processing is executed on GeoTIFF, but from a few tests it seems that GeoTIFF stores the data from the **LAST** timestamp, according to the defined temporal extent.

In this case study, we do not need to process GeoTIFF, but this format sometimes is just more suitable for a quick visualisation.

In [15]:
s2.download(full_tif_path, format="GTiff", validate=True)

# \ .filter_temporal(seasonal_extent) 
# TODO - to try batch jobs instead of downloading directly

[local_process] Total execution time (cumulative): 93.42 seconds


In [16]:
s2.download(full_nc_path, format="netcdf", validate=True)

[local_process] Total execution time (cumulative): 158.71 seconds


Let's check the spatial resolution in the downloaded netcdf (through external script):

In [17]:
import utils.inspect as inspect
inspect.check_res_nc(full_nc_path, ['B02', 'B05', 'SCL'])
stats = inspect.check_stats_nc(full_nc_path, ['B02', 'B05', 'SCL'])

Spatial resolution for band B02:
X: 10.0
Y: -10.0
Spatial resolution for band B05:
X: 10.0
Y: -10.0
Spatial resolution for band SCL:
X: 10.0
Y: -10.0
----------------------------------------
NetCDF Dataset Info:
----------------------------------------
Number of timestamps: 85
Time range: 2020-04-01T00:00:00.000000000 to 2020-10-30T00:00:00.000000000
B02 band shape: (85, 195, 95)
Stats:
  Nodata value: nan
  Mean: 3340.839599609375
  Min: -177.0
  Max: 18744.0
  Standard deviation: 3794.968505859375
----------------------------------------
B05 band shape: (85, 195, 95)
Stats:
  Nodata value: nan
  Mean: 3485.22998046875
  Min: -21.0
  Max: 16315.0
  Standard deviation: 3617.463134765625
----------------------------------------
SCL band shape: (85, 195, 95)
Stats:
  Nodata value: nan
  Mean: 7.101914882659912
  Min: 2.0
  Max: 10.0
  Standard deviation: 2.2306621074676514
----------------------------------------
[('B02', np.float32(-177.0), np.float32(nan)), ('B05', np.float32(-21.0), n

**The spatial resolution has changed at this step, bands with lower spatial resolution have been downsampled!**

TODO - could it be not downsampled? According to [documentation](https://open-eo.github.io/openeo-python-client/api.html#openeo.rest.datacube.DataCube.download), there are arguments `additional` and `job_options`. Might it work for keeping dimensions?

If SCL band does not have values <= 0 then no data value could be `nan`. Otherwise, we should count no data values to calculate the share of pixels that are going to be masked out.

Sometimes, Sentinel bands can have negative values of reflectance. For the sake of reliability, we should clamp them to 0 as missing/faulty/non-nominal values. For the details, see [ESA step forum](https://forum.step.esa.int/t/info-introduction-of-additional-radiometric-offset-in-pb04-00-products/35431).

Let's clamp negative values to 0:

In [34]:
# iterate over netcdf stats
ds = xr.open_dataset(full_nc_path)
for band_name, band_min, nodata_val in stats:
    if band_name in ds:
        print(f"Processing band {band_name}:")
        if band_min < 0: # remap to 0
            print(f"Band {band_name} has negative values. Remapping negative values to 0...")
            band_data = ds[band_name].values
            band_data[band_data < 0] = 0
            
            ds[band_name].values = band_data # update dataset with remapped values
            print(f"Updated {band_name} values and saved to NetCDF.")
        else:
            print(f"Band {band_name} has no negative values. No remapping needed.")
        print("-" * 40)        
ds.close()
ds.to_netcdf(full_nc_remap_path)
print(f"Remapped NetCDF has been saved to {full_nc_remap_path}.")
# NOTE - it seems like we cannot rewrite original 'full_nc_path' as Notebook stores open file handles

Processing band B02:
Band B02 has negative values. Remapping negative values to 0...
Updated B02 values and saved to NetCDF.
----------------------------------------
Processing band B05:
Band B05 has negative values. Remapping negative values to 0...
Updated B05 values and saved to NetCDF.
----------------------------------------
Processing band SCL:
Band SCL has no negative values. No remapping needed.
----------------------------------------
Remapped NetCDF has been saved to data/temp/full_remap.nc.
[local_process] Total execution time (cumulative): 191.28 seconds


Now the minimum value in bands (aside from SCL) should be 0:

In [35]:
stats = inspect.check_stats_nc(full_nc_remap_path, ['B02', 'B05', 'SCL'])

NetCDF Dataset Info:
----------------------------------------
Number of timestamps: 85
Time range: 2020-04-01T00:00:00.000000000 to 2020-10-30T00:00:00.000000000
B02 band shape: (85, 195, 95)
Stats:
  Nodata value: nan
  Mean: 3340.84228515625
  Min: 0.0
  Max: 18744.0
  Standard deviation: 3794.966796875
----------------------------------------
B05 band shape: (85, 195, 95)
Stats:
  Nodata value: nan
  Mean: 3485.23095703125
  Min: 0.0
  Max: 16315.0
  Standard deviation: 3617.46240234375
----------------------------------------
SCL band shape: (85, 195, 95)
Stats:
  Nodata value: nan
  Mean: 7.101914882659912
  Min: 2.0
  Max: 10.0
  Standard deviation: 2.2306621074676514
----------------------------------------
[local_process] Total execution time (cumulative): 191.70 seconds


Let's inspect the downloaded GeoTiff:

In [ ]:
import rasterio

with rasterio.open(full_tif_path) as tif:
    print("GeoTIFF Metadata:")
    print(tif.meta)  # Metadata information like crs, driver, number of bands, etc.
    print(tif.tags())  # Look for time-related metadata
    '''
    # Check number of bands
    print(f"Number of bands: {tif.count}")

    # Read the first band and print its shape
    band1 = tif.read(1)  # Reading the first band
    print(f"Shape of the first band: {band1.shape}")

    # Get CRS (Coordinate Reference System)
    print(f"CRS: {tif.crs}")


    # Calculate statistics (ignoring NaN values)
    band1_mean = np.nanmean(band1)
    band1_min = np.nanmin(band1)
    band1_max = np.nanmax(band1)
    band1_std = np.nanstd(band1)
    band1_sum = np.nansum(band1)
    band1_count = np.count_nonzero(~np.isnan(band1))  # Count of non-NaN pixels

    # Print the statistics
    print(f"Mean: {band1_mean}")
    print(f"Min: {band1_min}")
    print(f"Max: {band1_max}")
    print(f"Standard Deviation: {band1_std}")
    print(f"Sum (ignoring NaNs): {band1_sum}")
    print(f"Non-NaN count: {band1_count}")
    '''
print("-" * 40)

# Open the NetCDF file using xarray
'''
ds = xr.open_dataset(full_nc_remap_path)

# Print basic information about the NetCDF dataset
print("NetCDF Dataset Info:")

# Print the dimensions and variables of the dataset
print("Dimensions:")
print(ds.dims)
print("Variables:")
print(ds.variables)

# Check the temporal dimension (if present)
if 'time' in ds.dims:
    print(f"Number of time steps: {ds.dims['time']}")
    print(f"Time range: {ds['time'].values[0]} to {ds['time'].values[-1]}")

# Check a sample variable (for example, the first band)
if 'B02' in ds.variables:
    print(f"First band (B02) shape: {ds['B02'].shape}")

# Calculate statistics for the first band (B02), ignoring NaN values
if 'B02' in ds.variables:
    band_data = ds['B02'].values  # Extract the values as a numpy array

    # Calculate basic statistics, ignoring NaN values
    band_mean = np.nanmean(band_data)  # Mean, ignoring NaN
    band_min = np.nanmin(band_data)  # Min, ignoring NaN
    band_max = np.nanmax(band_data)  # Max, ignoring NaN
    band_std = np.nanstd(band_data)  # Standard deviation, ignoring NaN
    band_sum = np.nansum(band_data)  # Sum, ignoring NaN

    print(f"Statistics for Band B02:")
    print(f"  Mean: {band_mean}")
    print(f"  Min: {band_min}")
    print(f"  Max: {band_max}")
    print(f"  Standard Deviation: {band_std}")
    print(f"  Sum: {band_sum}")
'''
    

#### 4. Masking complicating pixels

The following mask concatenates all pixels occupied by [clouds](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/scene-classification/) in SCL band: 
- cloud shadows (3)
- unclassified (7)
- cloud medium probability (8)
- cloud high probability (9)
- thin cirrus (10)
- snow or ice (11)

Depending on the index processed, user can choose other combinations of values in SCL band.
**NOTE:** The code below is more about backend processing, not local! TODO - rewrite as local masking of full_nc_remap

In [37]:
scl_band = s2.band("SCL")
# TODO - to implement list of bands for the configuration

cloud_mask = ( (scl_band == 3) | (scl_band == 7) | (scl_band == 8) | (scl_band == 9) | (scl_band == 10) | (scl_band == 11)) * 1.0
'''cloud_mask = scl_band.isin([7,8,9,10,11]) * 1.0 # this doesn't work'''
cloud_mask.download(cloud_tif_path, format="GTiff", validate=True)
cloud_mask.download(cloud_nc_path, format="netcdf", validate=True)

# TODO - to print the share of masked values

'''# .filter_temporal('2020-07-01', '2020-07-07') \ '''

"# .filter_temporal('2020-07-01', '2020-07-07') \\ "

[local_process] Total execution time (cumulative): 355.42 seconds


Now, let's mask out the full cube by the mass with clouds:

~~TODO: It is still unclear why we are masking out the datacube by the mask from the last timestamp, not by mask with the matching timestamp.~~

In [38]:
masked_cloud = s2.mask(cloud_mask)

masked_cloud \
    .download(masked_cloud_tif_path, format="GTiff", validate=True)

# max_time() masks by the maximum value of time series. Otherwise, it will consider the whole datacube
# If we want to filter by the maximum timestamp:
'''masked_cloud \
    .max_time() \
    .download(masked_tif_path, format="GTiff", validate=True)'''
# This won't show the spatial resolution (dimension):!
# scl_band = s2.band("SCL")
# print(scl_band.metadata)



'masked_cloud     .max_time()     .download(masked_tif_path, format="GTiff", validate=True)'

[local_process] Total execution time (cumulative): 454.48 seconds


Let's check the spatial resolution of masked raster. It should be same across bands after masking:

In [39]:
inspect.check_res_tif(masked_cloud_tif_path)

Band 1 resolution: (10.0, 10.0)
Band 2 resolution: (10.0, 10.0)
Band 3 resolution: (10.0, 10.0)
----------------------------------------
[local_process] Total execution time (cumulative): 455.14 seconds


Save to masked netcdf and check the spatial resolution again:

In [40]:
masked_cloud \
    .download(masked_cloud_nc_path, format="netcdf", validate=True)
    # max_time() masks by the maximum value of time series, but we should mask by the corresponding timestamp

[local_process] Total execution time (cumulative): 545.22 seconds


TODO - Now, let's filter out timestamps that have too much masked values in the study area:

In [ ]:
...

Let's check the stats of output:

In [41]:
stats = inspect.check_stats_nc(masked_cloud_nc_path, ['B02', 'B05', 'SCL'])

NetCDF Dataset Info:
----------------------------------------
Number of timestamps: 85
Time range: 2020-04-01T00:00:00.000000000 to 2020-10-30T00:00:00.000000000
B02 band shape: (85, 195, 95)
Stats:
  Nodata value: nan
  Mean: 413.0081481933594
  Min: -177.0
  Max: 18744.0
  Standard deviation: 275.1376647949219
----------------------------------------
B05 band shape: (85, 195, 95)
Stats:
  Nodata value: nan
  Mean: 741.85595703125
  Min: -21.0
  Max: 4758.0
  Standard deviation: 494.0340270996094
----------------------------------------
SCL band shape: (85, 195, 95)
Stats:
  Nodata value: nan
  Mean: 4.741878509521484
  Min: 2.0
  Max: 6.0
  Standard deviation: 0.9086194634437561
----------------------------------------
[local_process] Total execution time (cumulative): 546.09 seconds


#### 5. Masking pixels of non-interest (water)
We should also mask out all pixels labeled as non-water after masking the data by clouds and snow:

In [ ]:
water_mask = (scl_band != 6) * 1.0 # all non-water pixels
water_mask.download(water_tif_path, format="GTiff", validate=True)
water_mask.download(water_nc_path, format="netcdf", validate=True)

Now, let's clip the **datacube masked by cloud** by non-water pixels:

In [ ]:
masked_cloud_water = masked_cloud.mask(water_mask)

masked_cloud_water \
    .download(masked_water_tif_path, format="GTiff", validate=True)

# if we want to filter datacube by only last timestamp
'''masked_cloud_water \
    .max_time() \
    .download(masked_water_tif_path, format="GTiff", validate=True)
    '''

And download as a datacube:

In [ ]:
masked_cloud_water \
    .download(masked_water_nc_path, format="netcdf", validate=True)

Check the spatial resolution again (through external script):

In [ ]:
inspect.check_res_nc(masked_cloud_nc_path, ['B02', 'B05', 'SCL'])
inspect.check_res_nc(masked_water_nc_path, ['B02', 'B05', 'SCL'])

Let's explore the netcdf output. If everything is correct, you will wind three coordinates (x, y and time), four variables (bands 2, 5, SCL and coordinate reference system), as well as three indices for each dimension. Currently, CRS variable is not inheriting the original CRS of OpenEO collection.

In [ ]:
x = xr.open_dataset(full_nc_remap_path)
x

'''
# check no data values
for var in x.data_vars:
    nodata_val = x[var].rio.nodata
    print(nodata_val)
'''
'''
# Initialize counter for NoData values
total_nodata = 0

# Loop through each variable and count NoData values
for var in x.data_vars:
    nodata_value = x[var].rio.nodata
    if nodata_value is not None:
        total_nodata += (x[var] == nodata_value).sum().item()

print(f"Total NoData values in dataset: {total_nodata}")
'''

We should repoject GeoTIFF outputs to illustrate them:

In [ ]:
# reproject tiff files to illustrate them (external script)
from utils.reproject import VectorProc
raster_reproj = VectorProc(full_tif_path).reproject_Ras2Leaflet(full_tif_reproj_path)
raster_reproj = VectorProc(masked_cloud_tif_path).reproject_Ras2Leaflet(masked_cloud_tif_reproj_path)
raster_reproj = VectorProc(masked_water_tif_path).reproject_Ras2Leaflet(masked_water_tif_reproj_path)

Let's visualise intermediate results through GeoTIFF:

In [ ]:
# TODO - somehow reprojected layers are not being posted on Leaflet map!
sm = leafmap.Map(center=center, zoom=zoom)
vector.explore(m=m, style_kwds=dict(fill=True, color='blue', fill_opacity=0.5))
vector.convex_hull.explore(m=m, style_kwds=dict(fill=False, color='green'))
bounding_box_df.explore(m=m, style_kwds=dict(color="black", fill = False)) # or style_kwds={"color": "black", "fillOpacity": 0}

m.add_raster(full_tif_reproj_path, bands=[1,2,3], layer_name = "colour") # put band numbers, not their names (like B02,B05 or RGB)
m.add_raster(masked_cloud_tif_reproj_path, bands=[1,2,3], layer_name = "masked_by_clouds")
m.add_raster(masked_water_tif_reproj_path, bands=[1,2,3], layer_name = "masked_by_cloudsNwater")
m # show

# issue met on Windows with conda env: ImportError: localtileserver is not installed. Please install it before proceeding. https://github.com/banesullivan/localtileserver
# localtileserver on Windows is problematic, so successfully using Docker - Ubuntu-based image for osgeo plus requirements.

Now, let's add the cloud mask as well:

In [ ]:
m.add_raster(cloud_tif_path, bands=[1], layer_name = "cloud")

#### 6. Calculate NDTrI index
According to previous experiments [put link](link.ccomm), the trophic state of water in lakes can be defined through the normalised difference between B2 (red) and B5 (red edge).

To calculate it, we should just put a simple math expression: 
$$ \text{NDTrI} = \frac{B05-B02}{B05+B02} $$

Now, let's process it locally on user's machine just for the sample GeoTIFF output to see if everything is working fine:

In [ ]:
# Local processing
import rasterio

# TODO - to include resampling

# open local masked output
with rasterio.open(masked_cloud_tif_path) as dataset:
    # read bands
    b02_data = dataset.read(1)  # B2
    b05_data = dataset.read(2)  # B5
    # raster algebra
    ndtri = (b05_data - b02_data) / (b05_data + b02_data)
    '''# handle division by zero (where both bands are zero)
    ndtri[np.isnan(ndtri)] = 0  # optional - replace NaN with 0'''
    # plot array
    plt.imshow(ndtri, cmap='viridis')
    plt.colorbar()
    plt.title("Normalised Difference Trophic Index (NDTrI)")
    plt.show()
    
    # update metadata
    meta = dataset.meta
    meta.update(dtype=rasterio.float32, count=1)
    
    # to file
    with rasterio.open(ndtri_tif_path, "w", **meta) as out_dataset:
        out_dataset.write(ndtri, 1)
    

In [ ]:
'''min_level = min(s2.band("B01"), s2.band("B02"), s2.band("B03"))'''

# expression to run on the backend
ndtri = (s2.band("B05") - s2.band("B02")) / (s2.band("B05") + s2.band("B02"))
ndtri # these illustrations of objects are not always visualised

**Local processing**

Next step is to clip the NDTrI by the polygon of interest. For the sake of clarity, reprojection process is defined in the [external script](utils/reproject.py):

In [ ]:
from utils.reproject import VectorProc
import rasterio
from rasterio.mask import mask

vector_proc = VectorProc(ndtri_tif_path)
vector_reproj = vector_proc.reproject_vec2ras(vector)

with rasterio.open(ndtri_tif_path) as dataset:
    # mask
    first_feature = vector_reproj.iloc[0:1]
    geometries = first_feature.geometry.values
    out_image, out_transform = mask(dataset, geometries, crop=True)
    out_meta = dataset.meta.copy()    # get metadata for output
    # update metadata to reflect the new image size after masking
    out_meta.update({"driver": "GTiff",
                     "count": 1,
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

with rasterio.open(masked_ndtri_tif_path, "w", **out_meta) as out:
    out.write(out_image)

print(f"Masked raster saved to {masked_ndtri_tif_path}")

Now, it's time to mask the NDTrI by clouds:

In [ ]:
ndtri_masked = ndtri.mask(cloud_mask) # replacement is null by default
ndtri

#### 7. Mask Polygon: From Bounding Box to Shape
We have a cloud masked snow map data cube now. In order to keep only pixels within the exact chatchment boundaries we mask to the outline of the catchment. Values outside of the boundaries are set to NA.

In [ ]:
vector_border = vector['geometry'].iloc[0]
ndtri_masked_cut = ndtri_masked.mask_polygon(vector_border)
ndtri_masked_cut

##### **Visualisation**
Let's have a first look at a time slice of our NDTrI map. So far we have not computed anything. We have only defined a set of functions that are going to be applied in sequence. This makes up our workflow or processing graph. 
To reduce the data volume which we are going to download we are only selecting one time step of our data cube.

**In order to start the processing we have to tell the cloud platform specifically that we want to execute our workflow. In this case we want to start the processing directly without registering a job on the backend. This solution is good for small amounts of data. For larger processing tasks batch jobs are preferred (we'll do that later).**

In [ ]:
ndtri_masked_cut_short  = ndtri_masked_cut.filter_temporal(temporal_extent)
ndtri_masked_cut_short.download(test_nc_path, format="netcdf", validate=True)
ndtri_masked_cut_short.download(test_tif_path, format="GTiff", validate=True)

Let's inspect the output content in read-binary mode:

In [ ]:
with open(test_nc_path, "rb") as f: 
    print(f.read(100))

Once the processing is done on the cloud and the data is downloaded we can load the file into our working environment and plot it!

TODO - to check
The area of interest is spread across two S2 tiles. This is visibile in the northern part of the plot because we chose one specific acquisition date where there is not data available for the northern tile.

Now, let's open the output xarray and find the its length (number of timestamps available). We can also check the dimensions (or coordinates) of xarray:

In [ ]:
x = xr.open_dataarray(test_nc_path,decode_coords="all",engine="rasterio")
len(x)
print(x.coords)

Let's visualise on of the datacube slices (timestamps):

TODO - to find out how CRS is encoded in xarray metadata as currently not available:

In [ ]:
import utils.visual as visual

visual.Visual(slice = x[0]).map()

"""
fig, ax = plt.subplots(figsize=(8, 6))
im = slice.plot.imshow(cmap='viridis', ax=ax)
cbar = im.colorbar #add colourbar
cbar.set_label("Normalised Difference Trophic Index (NDTrI)") 

timestamp = slice.coords['t'].values
timestamp_str = str(timestamp)

# axes
ax.set_title(f"Normalised Difference Trophic Index (NDTrI), t = {timestamp_str}\n CRS = {crs}")
ax.set_xlabel("X coordinate")
ax.set_ylabel("Y coordinate")
"""

That was just the first timestamp, so let's take another one.

**Attention**: Islands are not covered by this datacube, because we have already filtered out non-water pixels.
TODO - to reproject output!

In [ ]:
visual.Visual(slice = x[3]).map()

If we try to visialise the datacube slice with number > array length, we face an error:

In [ ]:
visual.Visual(slice = x[1000]).map()

Some pixels are not consistenly being defined as water ones.

In [ ]:
visual.Visual(slice = x[6]).map()

Another example:

In [ ]:
visual.Visual(slice = x[7]).map()

Sometimes we would like to exclude coastal areas of lake with outlying NDTrI values.
Let's try to apply the negative buffer to the borderline of lake if do not want to explore the coastal areas:

In [ ]:
import scipy.ndimage as ndi

#mask - all values are features of interest, except from no data
mask = ~np.isnan(x) # invert values
# compute euclidean distance to the the mask, and apply negative buffer
distance = ndi.distance_transform_edt(mask)  # distance to mask (background)
if shore_negative_buffer:
    # only keep values within the negative buffer
    buffered_mask = distance > shore_negative_buffer
    # apply the buffered mask to original data (all values outside the buffer are None)
    buffered_slice = x.where(buffered_mask)
else:
    warnings.warn("Negative buffer has not been defined...")

# visualise
visual.Visual(buffered_slice[6]).map()

Let's visualise on OpenStreetMap:

In [ ]:
raster_reproj = VectorProc(test_tif_path).reproject_Ras2Leaflet(test_tif_reproj_path)

# m = leafmap.Map(center=center, zoom=15)
m = leafmap.Map(layers_control=True)
# lake.convex_hull.explore(m=m)
m.add_raster(test_tif_reproj_path, indexes=[0], palette="coolwarm", layer_name="ndtri_masked_clipped")
m # show

# TODO - layers are not displayed: https://localtileserver.banesullivan.com/installation/remote-jupyter.html
# seems like setting client prefix doesn't help

**NOTE**: Regarding the rule on removing pixels with the seasonal share of water less than 80% (recognised by Sentinel as non-water at more than 20% of timestamps), we do not need to implement it generally. So, each pixel at each timestamp is defined in a binary mode, either water or non-water.

#### **8. Aggregation**

How much time did it take?

In [ ]:
track_time.get_time("local_process")

## ***Processing on the backend***
In this chapter, we will access data on data cube, but run processing mainly on the back-end.
As you have seen, we have been executing OpenEO processes and then downloading the temporary outputs on the local drive.
Now, what if we could try to define OpenEO processes, but do not download the temporary files and run processing on the OpenEO server?

....

### Register a batch job for processing

TODO - to rewrite
We are starting the processing now with a batch job. This registers our job on the backend in our user space and assigns further information to the job, such as an ID, the job status, the process graph and further metadata. First we specifiy the end of our process graph with `save_result()` and specifiy the format (since we aggregated over the spatial dimension we will receive three arrays of data. So JSON is a suitable format). Then we create the batch job and start it.

In [ ]:
job = cube.create_job(title="NDTrI timeseries 2020")

# Define the end of the process graph and the output format
n_pixels_json = n_pixels.save_result(format="JSON")
# Create a batch job
job = n_pixels_json.create_job(title="n_pixels_json")
# start the job and wait till it finishes
job.start_and_wait()

Now we can check the status of our job. We can download the result once the job has finished.

In [ ]:
job.status()

In [ ]:
if job.status() == "finished":
    results = job.get_results()
    results.download_files(timeseries_path)

**Quick hint: take a look at the job description: e.g. `job.describe_job()`**

### Load the resulting time series
Let's load the result. It contains the total number of pixels in the catchment, number of cloud and snow pixels.

In [ ]:
# load the result
with open(timeseries_path,"r") as file:
    n_pixels_json = json.load(file)

In [ ]:
# check the first 5 entries to check the data structure.
list(n_pixels_json.items())[:3] # careful unsorted dates due to JSON format

**_Quick hint: what is the length of the time series JSON?_**
`len(n_pixels_json)`

Now we do some data wrangling to get a structured data frame.

In [ ]:
# Create a Pandas DataFrame to hold the values
dates = [k for k in n_pixels_json]
n_catchment_vals = [n_pixels_json[k][0][0] for k in n_pixels_json]
n_cloud_vals = [n_pixels_json[k][0][1] for k in n_pixels_json]
n_snow_vals = [n_pixels_json[k][0][2] for k in n_pixels_json]

data = {
        "time":pd.to_datetime(dates),
        "n_catchment_vals":n_catchment_vals,
        "n_cloud_vals":n_cloud_vals,
        "n_snow_vals":n_snow_vals
       }
df = pd.DataFrame(data=data).set_index("time")
# Sort the values by date
df = df.sort_values(axis=0,by="time")
df[:3]

In [ ]:
df.n_snow_vals.sum()

### Calculate the cloud percentage for filtering time steps
Divide the number of cloudy pixels by the number of total pixels = cloud percentage

In [ ]:
perc_cloud = df["n_cloud_vals"].values / df["n_catchment_vals"].values * 100
df["perc_cloud"] = perc_cloud
df[:3]

**Quick hint: The sum of the n_catchment_vals should give an overall idea of the total number of pixels in the datacube for the whole time-series** `df.n_catchment_vals.sum()`

**Quick hint: a filter of the snow values can give an idea of when the maximum snow cover occurred** `df.where(df.n_snow_vals == df.n_snow_vals.max())`

**Quick hint: a simplified approach for converting from pixel count to square kilometres is to use this simplified formula::**

${{Area (km^2)} = (\frac{Spatial resolution (meters/pixel)^2}{1,000,000})\times{\text{Total pixel count}}}$

Plot the timeseries and the cloud threshold of 20%. If the cloud cover is higher the timestamp will be excluded later on.

Plot the **cloud percentage** with the threshold.

In [ ]:
df.plot(y="perc_cloud",rot=45,kind="line",marker='o')
plt.axhline(y = 25, color = "r", linestyle = "-")
plt.show()

### Calculate the snow percentage
Divide the number of snow pixels by the number of total pixels = snow percentage

In [ ]:
perc_snow = df["n_snow_vals"].values / df["n_catchment_vals"].values * 100
df["perc_snow"] = perc_snow
df[:3]

Plot the **unfiltered snow percentage**

In [ ]:
df.plot(y="perc_snow",rot=45,kind="line",marker='o')
plt.show()

### Filter out cloudy time steps
Keep only the dates with cloud coverage less than the threshold

In [ ]:
df_filtered = df.loc[df["perc_cloud"]<25]

### Plot and save the cloud free snow percentage time series
Plot the **cloud filtered snow percentage**

In [ ]:
df_filtered.plot(y="perc_snow",rot=45,kind="line",marker='o')
plt.show()

Save the **cloud filtered snow percentage**

In [ ]:
df_filtered.to_csv("31_results/filtered_snow_perc.csv")

In [ ]:
pip install cdsapi

In [ ]:
import cdsapi

dataset = "cams-europe-air-quality-reanalyses"
request = {
    'variable': ['particulate_matter_2.5um'],
    'model': ['ensemble'],
    'level': ['0'],
    'type': ['validated_reanalysis'],
    'year': ['2021'],
    'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


Now, let's just check how much time it will take to download a datacube with the same parameters, but for the larger area:

And print statictics in graphical form:

## ***Batch processing on the backend***